In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

In [35]:
accounts_df = pd.read_csv('account.csv', sep=';')
cards_df = pd.read_csv('card.csv', sep=';')
clients_df = pd.read_csv('client.csv', sep=';')
dispos_df = pd.read_csv('disp.csv', sep=';')
district_df = pd.read_csv('district.csv', sep=';')
loan_df = pd.read_csv('loan.csv', sep=';')
order_df = pd.read_csv('order.csv', sep=';')
trans_df = pd.read_csv('trans.csv', sep=';', engine='python')

In [24]:
def date_correction(df, col_name):
  """ Function that will re-format cells into date format. Input should be in the format of YYMMDD, e.g. 950107
  df: dataframe
  col_name: column name of the df
  return: updated df
  """
  df[col_name] = pd.to_datetime(df[col_name], format='%y%m%d', errors='coerce')
  return df

# correct the year when 45 being mistaken as 2045 and not 1945
def date_typo_correction(date): 
  """Function that corrects the year when 45 being mistaken as 2045 and not 1945
  date: date with '%y%m%d' format
  return: corrected date"""
  if date.year > 2000: 
      date = date.replace(year=date.year-100)
      return date
  else:
      return date

def calculate_age(born_date):
  """Age calculation as of 31.12.1998 
  born_date: date with '%y%m%d' format
  return: age(int)"""
  born = born_date.year
  return 1998 - born

**Accounts**

In [ ]:
accounts_df.head()

,account_id,district_id,frequency,date
0,576,55,POPLATEK MESICNE,930101
1,3818,74,POPLATEK MESICNE,930101
2,704,55,POPLATEK MESICNE,930101
3,2378,16,POPLATEK MESICNE,930101
4,2632,24,POPLATEK MESICNE,930102


In [4]:
acc_out = accounts_df.copy()

# date column for accounts_df refers to "Date of account creation", given it 
# lacks of relevance, it will be dropped
acc_out = acc_out.drop(columns=['date'])

freq_cz_to_en = {'POPLATEK MESICNE': 'Monthly Issuance', 'POPLATEK TYDNE': 'Weekly Issuance', 'POPLATEK PO OBRATU': 'Issuance after Transaction'}
acc_out['frequency'] = acc_out['frequency'].replace(freq_cz_to_en)
acc_out.head()

,account_id,district_id,frequency
0,576,55,Monthly Issuance
1,3818,74,Monthly Issuance
2,704,55,Monthly Issuance
3,2378,16,Monthly Issuance
4,2632,24,Monthly Issuance


**Card**

In [ ]:
cards_out = cards_df.copy()
cards_out['issued'] = pd.to_datetime(cards_out['issued'], errors='coerce')
cards_out.head()

,card_id,disp_id,type,issued
0,1005,9285,classic,1993-11-07
1,104,588,classic,1994-01-19
2,747,4915,classic,1994-02-05
3,70,439,classic,1994-02-08
4,577,3687,classic,1994-02-15


**Client**

In [ ]:
clients_out = clients_df.copy()
clients_out.head()

,client_id,birth_number,district_id
0,1,706213,18
1,2,450204,1
2,3,406009,1
3,4,561201,5
4,5,605703,5


According to the documentation of the dataset, for column "**birth_number**":

"The number is in the form YYMMDD for men,

the number is in the form YYMM+50DD for women,
where YYMMDD is the date of birth"

In [ ]:
# adding new columns 'gender' and 'age'
clients_out = clients_out.join(pd.DataFrame({'birth_date': np.nan, 'gender': np.nan, 'age': np.nan}, index = clients_df.index))

# select rows with YYMM+50DD calculate age, mark as female, repeat same process for man using normal date time YYMMDD
clients_out['birth_date'] = clients_out ['birth_number']
for ids, item in enumerate (clients_out ['birth_number']):
    if int(str(item)[2:4]) > 50:
        clients_out.loc[ids, 'gender'] = 0 #female
        clients_out.loc[ids, 'birth_date'] = item - 5000 
    else: 
        clients_out.loc[ids, 'gender'] = 1

In [ ]:
#converting birth_date into date format
clients_out = date_correction(clients_out, 'birth_date') 

#correcting misinterpreted dates
clients_out['birth_date'] = clients_out['birth_date'].apply(date_typo_correction)

# calculate Age
clients_out['age'] = clients_out['birth_date'].apply(calculate_age)

# Remove 'birth_number' column
clients_out = clients_out.drop(['birth_number'], axis=1)
clients_out.head()

,client_id,district_id,birth_date,gender,age
0,1,18,1970-12-13,0.0,28
1,2,1,1945-02-04,1.0,53
2,3,1,1940-10-09,0.0,58
3,4,5,1956-12-01,1.0,42
4,5,5,1960-07-03,0.0,38


**Disposition**

In [ ]:
dispos_out = dispos_df.copy()
dispos_out.head()

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT


### **District**

In [36]:
demographic_out = district_df.copy()
demographic_out.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


In [37]:
demo_mapping = {'A1': 'district_id',
                 'A2': 'district_name',
                 'A3': 'region_name',
                 'A4': 'inhabitants_number',
                 'A5': 'number_of_municipalities_with_<499_inhabitants',
                 'A6': 'number_of_municipalities_with_500-1499_inhabitants',
                 'A7': 'number_of_municipalities_with_2000-9999_inhabitants',
                 'A8': 'number_of_municipalities_with_>10000_inhabitants',
                 'A9': 'city_number',
                 'A10': 'urban_inhabitants_ratio',
                 'A11': 'average salary',
                 'A12': '1995_unemployment_rate',
                 'A13': '1996_unemployment_rate',
                 'A14': 'entrepreneurs_per_1000_inhabitants',
                 'A15': 'crimes_committed_number_in_1995',
                 'A16': 'crimes_committed_number_in_1996'
                 }

demographic_out = demographic_out.rename(columns=demo_mapping)
demographic_out.head()

,district_id,district_name,region_name,inhabitants_number,number_of_municipalities_with_<499_inhabitants,number_of_municipalities_with_500-1499_inhabitants,number_of_municipalities_with_2000-9999_inhabitants,number_of_municipalities_with_>10000_inhabitants,city_number,urban_inhabitants_ratio,average salary,1995_unemployment_rate,1996_unemployment_rate,entrepreneurs_per_1000_inhabitants,crimes_committed_number_in_1995,crimes_committed_number_in_1996
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.29,0.43,167,85677,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.67,1.85,132,2159,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.95,2.21,111,2824,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.64,5.05,109,5244,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.85,4.43,118,2616,3040


**Loan**

In [ ]:
loan_out = loan_df.copy()
loan_out.head()

,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033.0,B
1,5316,1801,930711,165960,36,4610.0,A
2,6863,9188,930728,127080,60,2118.0,A
3,5325,1843,930803,105804,36,2939.0,A
4,7240,11013,930906,274740,60,4579.0,A


In [ ]:
loan_out['status_desc'] = loan_out ['status']

loan_status_desc = {'A':'Contract finished, no problem', 
                'B':'Contract finised, loan not paid',
                'C':'Runing contract, OK so far',
                'D':'Runing contract, client in debt'
              }
loan_out['status_desc'] = loan_out['status_desc'].replace(loan_status_desc)

loan_out ['status_numeric'] = loan_out['status']

# represent bad loans as -1 and good ones as 1 
loan_status_num = {'A':1, 'B':-1, 'C':1, 'D':-1}
loan_out.status_numeric = loan_out.status_numeric.replace(loan_status_num)

# correct date
loan_out = date_correction(loan_out, 'date') 
loan_out.head()

,loan_id,account_id,date,amount,duration,payments,status,status_desc,status_numeric
0,5314,1787,1993-07-05,96396,12,8033.0,B,"Contract finised, loan not paid",-1
1,5316,1801,1993-07-11,165960,36,4610.0,A,"Contract finished, no problem",1
2,6863,9188,1993-07-28,127080,60,2118.0,A,"Contract finished, no problem",1
3,5325,1843,1993-08-03,105804,36,2939.0,A,"Contract finished, no problem",1
4,7240,11013,1993-09-06,274740,60,4579.0,A,"Contract finished, no problem",1


**Order**

In [7]:
order_out = order_df.copy()
order_out.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,


In [8]:
# translate Czech to English
symbol_map_order = {'POJISTNE': 'insurrance_pmt', 'UVER': 'loan_pmt', 'SIPO': 'household', 'LEASING': 'leasing'}
order_out['purpose'] = order_out['k_symbol'].replace(symbol_map_order)
order_out.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol,purpose
0,29401,1,YZ,87144583,2452.0,SIPO,household
1,29402,2,ST,89597016,3372.7,UVER,loan_pmt
2,29403,2,QR,13943797,7266.0,SIPO,household
3,29404,3,WX,83084338,1135.0,SIPO,household
4,29405,3,CD,24485939,327.0,,


In [9]:
# the empty cells under k_symbol is string with one blank space
order_out.k_symbol[4]

' '

In [10]:
order_out = order_out.replace(' ', np.nan)
order_out.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol,purpose
0,29401,1,YZ,87144583,2452.0,SIPO,household
1,29402,2,ST,89597016,3372.7,UVER,loan_pmt
2,29403,2,QR,13943797,7266.0,SIPO,household
3,29404,3,WX,83084338,1135.0,SIPO,household
4,29405,3,CD,24485939,327.0,NaN,NaN


**Trans**

In [19]:
trans_out = trans_df.copy()
trans_out.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,930101,PRIJEM,VKLAD,600.0,600.0,NaN,NaN,NaN
4,579373,1972,930102,PRIJEM,VKLAD,400.0,400.0,NaN,NaN,NaN


In [20]:
# Blank space also exists in "k_symbol"
for i in ['type', 'operation', 'k_symbol']:
  print(f'{i} has unique value: {trans_out[i].unique()}')

type has unique value: ['PRIJEM' 'VYDAJ' 'VYBER']
operation has unique value: ['VKLAD' 'PREVOD Z UCTU' 'VYBER' nan 'PREVOD NA UCET' 'VYBER KARTOU']
k_symbol has unique value: [nan 'DUCHOD' 'UROK' 'SIPO' 'SLUZBY' ' ' 'POJISTNE' 'SANKC. UROK' 'UVER']


In [21]:
# Replace blank space with "NaN"
trans_out = trans_out.replace(' ', np.nan)

In [25]:
# correct the dates
trans_out = date_correction(trans_out, 'date')

type_map = {'PRIJEM': 'credit', 'VYDAJ': 'withdrawal'}
op_map = {'VYBER KARTOU': 'credit_card_withdrawal', 
          'VKLAD': 'deposit', 
          'PREVOD Z UCTU': 'collection_from_another_bank', 
          'VYBER': 'withdraw_cash',
          'PREVOD NA UCET': 'transer_to_another_bank'}
symbol_map_trans = {'POJISTNE': 'insurrance_pmt', 
                    'UVER': 'loan_pmt', 
                    'SIPO': 'household', 
                    'SLUZBY': 'service',
                    'UROK': 'interest_pmt',
                    'SANKC. UROK': 'interest_for_negetive_balance',
                    'DUCHOD': 'pension'}
trans_out['type'] = trans_out['type'].replace(type_map)
trans_out['operation'] = trans_out['operation'].replace(op_map)
trans_out['k_symbol'] = trans_out['k_symbol'].replace(symbol_map_trans)
trans_out.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,1993-01-01,credit,deposit,700.0,700.0,NaN,NaN,NaN
1,171812,576,1993-01-01,credit,deposit,900.0,900.0,NaN,NaN,NaN
2,207264,704,1993-01-01,credit,deposit,1000.0,1000.0,NaN,NaN,NaN
3,1117247,3818,1993-01-01,credit,deposit,600.0,600.0,NaN,NaN,NaN
4,579373,1972,1993-01-02,credit,deposit,400.0,400.0,NaN,NaN,NaN


In [26]:
# df where operation has missing value
op_empty_df = trans_out[trans_out['operation'].isnull()][trans_out.columns]
op_empty_df.groupby('type', axis=0).count()

,trans_id,account_id,date,operation,amount,balance,k_symbol,bank,account
type,,,,,,,,,
credit,183114,183114,183114,0,183114,183114,183114,0,0


In [27]:
op_empty_df.groupby('k_symbol', axis=0).count()

,trans_id,account_id,date,type,operation,amount,balance,bank,account
k_symbol,,,,,,,,,
interest_pmt,183114,183114,183114,183114,0,183114,183114,0,0


All missing value type are all interest payment, so I replaced the missing value in 'Operation' with 'interest_pmt'.

In [28]:
#substituting missing values in "operation"
trans_out['operation'].fillna('interest_pmt', inplace = True)
trans_out.isnull().sum()

trans_id           0
account_id         0
date               0
type               0
operation          0
amount             0
balance            0
k_symbol      535314
bank          782812
account       760931
dtype: int64

In [38]:
acc_out.to_csv('clean_account.csv', index=False)
cards_out.to_csv('clean_card.csv', index=False)
clients_out.to_csv('clean_client.csv', index=False)
dispos_out.to_csv('clean_disp.csv', index=False)
demographic_out.to_csv('clean_district.csv', index=False)
loan_out.to_csv('clean_loan.csv', index=False)
order_out.to_csv('clean_order.csv', index=False)
trans_out.to_csv('clean_trans.csv', index=False)